In [28]:
from docx import Document
from pandas import DataFrame, concat
import pandas as pd
import re

In [29]:
def read_docx(file_path):
    doc = Document(file_path)
    return doc

In [30]:
def read_docx(file_path):
    doc = Document(file_path)
    full_text = []
    for para in doc.paragraphs:
        full_text.append(para.text)
    return "\n".join(full_text)


In [31]:
file_path = "../data/test.docx"
text = read_docx(file_path)
print(text)

test[dʒɪ]
[dʒɪ]ficulda[dʒɪ]!
[dʒɪ]feren[tʃɪ].
inconscien[tɪ]men[tɪ],
tris[te]men[te]?	
inocen[te]men[tɪ]-a	
es[tɪ]ve[ɹ(v)][de][s]-o
tris[te]mente
[de]
[dɪ]
tes[te]….


In [32]:
def df_find_cases_without_duplication(file_path):
    words_for_table = []  # Use a list to store repeated entries
    text = read_docx(file_path)
    # Use regular expression to find words with 't' inside brackets but without 'ʃ' right after it
    
    # padroes de patalizações
    pattern = r'\S*\[[^\]]*(?:tʃ|t(?!ʃ)|dʒ|de|dɪ)[^\]]*\]\S*'

    
    matches = re.findall(pattern, text)
    

    words_for_table.extend(matches)
    
    
    df = DataFrame(words_for_table, columns=['palavra'])
    
    #remove commas and periods at the end of values
    punctuation_pattern = r'[.,…!?]+$'
    a_pattern = r'-a'
    o_pattern = r'-o'
    pattern = punctuation_pattern + '|' + a_pattern + '|' + o_pattern
    df['palavra'] = df['palavra'].str.replace(pattern, '', regex=True)
        
            
    return df

file_path = "../data/test.docx"
df = df_find_cases_without_duplication(file_path)
df

,palavra
0,test[dʒɪ]
1,[dʒɪ]ficulda[dʒɪ]
2,[dʒɪ]feren[tʃɪ]
3,inconscien[tɪ]men[tɪ]
4,tris[te]men[te]
5,inocen[te]men[tɪ]
6,es[tɪ]ve[ɹ(v)][de][s]
7,tris[te]mente
8,[de]
9,[dɪ]


In [33]:
def create_patalizacao(row):
    
    pattern_t = r'\[[^\]]*t(?!ʃ)[^\]]*\]'
    pattern_tf = r'\[[^\]]*tʃ[^\]]*\]'
    pattern_dz = r'\[[^\]]*dʒ[^\]]*\]'
    pattern_de = r'\[[^\]]*de[^\]]*\]'
    pattern_di = r'\[[^\]]*dɪ[^\]]*\]'

    pattern = pattern_t + '|' + pattern_tf + '|' + pattern_dz + '|' + pattern_de + '|' + pattern_di
    

    patalizacao_cru = re.findall(pattern, row['palavra'])
    
    

    if re.search(pattern_t, patalizacao_cru[0]):
        patalização = '[t]'
    if re.search(pattern_tf, patalizacao_cru[0]):
        patalização ='[tʃ]'
    if re.search(pattern_dz, patalizacao_cru[0]):
        patalização = '[dʒ]'
    if re.search(pattern_de, patalizacao_cru[0]):
        patalização = '[de]'
    if re.search(pattern_di, patalizacao_cru[0]):
        patalização = '[dɪ]'

    return patalizacao_cru, patalização

In [34]:
# Function to determine 'sonoridade' based on 'patalização' values
def determine_sonoridade(row):
    
    if row == '[t]' or row == '[tʃ]':
        return 'surda'
    elif row == '[de]' or row == '[dɪ]' or row == '[dʒ]':
        return 'sonora'
    
    return None


In [35]:

def create_vogais_alvo(row):
    vogais_alvo = []
    
    pattern_I = r'\[[^\]]*ɪ[^\]]*\]'
    pattern_e = r'\[[^\]]*e[^\]]*\]'
    pattern = pattern_I + '|' + pattern_e

    match = re.search(pattern, row['palavra'])

    if match:
        vogais_cru = match.group(0)

        if re.search(pattern_I, vogais_cru):
            return '[ɪ]'
        if re.search(pattern_e, vogais_cru):
            return '[e]'
        
    return None
    
    

In [36]:
def apply_cols(file_path):
    df = df_find_cases_without_duplication(file_path)
    df['patalizacao_cru'], df['patalização'] = zip(*df.apply(create_patalizacao, axis=1))
    # df['patalização_count'] = df['patalização'].apply(lambda x: len([item.strip() for item in x.split(',')]))
    df['sonoridade'] = df['patalização'].apply(determine_sonoridade)
    df['vogais_alvo'] = df.apply(create_vogais_alvo, axis=1)
    return df

In [37]:
df = apply_cols(file_path)
df

,palavra,patalizacao_cru,patalização,sonoridade,vogais_alvo
0,test[dʒɪ],[[dʒɪ]],[dʒ],sonora,[ɪ]
1,[dʒɪ]ficulda[dʒɪ],"[[dʒɪ], [dʒɪ]]",[dʒ],sonora,[ɪ]
2,[dʒɪ]feren[tʃɪ],"[[dʒɪ], [tʃɪ]]",[dʒ],sonora,[ɪ]
3,inconscien[tɪ]men[tɪ],"[[t], [tɪ]]",[t],surda,[ɪ]
4,tris[te]men[te],"[[t], [te]]",[t],surda,[e]
5,inocen[te]men[tɪ],"[[t], [tɪ]]",[t],surda,[e]
6,es[tɪ]ve[ɹ(v)][de][s],"[[t], [de]]",[t],surda,[ɪ]
7,tris[te]mente,[[t]],[t],surda,[e]
8,[de],[[de]],[de],sonora,[e]
9,[dɪ],[[dɪ]],[dɪ],sonora,[ɪ]


In [38]:
def create_posicao_patalizacao_final(row):
    
    
    unico_pattern = r'^\[[^\]]*(t(?!ʃ)|tʃ|dʒ|de|dɪ)[[^\]]*\]$'

  
    comeco_pattern = r'^\[[^\]]*(t(?!ʃ)|tʃ|dʒ|de|dɪ)[^\]]*\]'

   
    meio_pattern = r'\[[^\]]*(tʃ|t(?!ʃ)|dʒ|de|dɪ)[^\]]*\]'
    meio_pattern_mais = r'\S\[[^\]]*(?:tʃ|t(?!ʃ)|dʒ|de|dɪ)[^\]]*\]\S'


    final_pattern = r'\[[^\]]*(t(?!ʃ)|tʃ|dʒ|de|dɪ)[^\]]*\]$'


    

    row['copy_palavra'] = row['palavra']

    if re.search(unico_pattern, row['palavra']):
        row['posição_patalização'] = row['patalização'] + 'único'
        row['copy_palavra'] = re.sub(unico_pattern, '', row['copy_palavra'])

    elif re.search(comeco_pattern, row['palavra']):
        row['posição_patalização'] = row['patalização'] + ' comeco'
        row['copy_palavra'] = re.sub(comeco_pattern, '', row['copy_palavra'])

    elif re.search(meio_pattern_mais, row['palavra']):
        row['posição_patalização'] = row['patalização'] + ' meio'
        row['copy_palavra'] = re.sub(meio_pattern, '', row['copy_palavra'], count=1)
    
    elif re.search(final_pattern, row['palavra']):
        row['posição_patalização'] = row['patalização'] + ' final'
        row['copy_palavra'] = re.sub(final_pattern, '', row['copy_palavra'])
    
    else:
        row['posição_patalização'] = ''
    return row


df = df.apply(create_posicao_patalizacao_final, axis=1)
df


,palavra,patalizacao_cru,patalização,sonoridade,vogais_alvo,copy_palavra,posição_patalização
0,test[dʒɪ],[[dʒɪ]],[dʒ],sonora,[ɪ],test,[dʒ] final
1,[dʒɪ]ficulda[dʒɪ],"[[dʒɪ], [dʒɪ]]",[dʒ],sonora,[ɪ],ficulda[dʒɪ],[dʒ] comeco
2,[dʒɪ]feren[tʃɪ],"[[dʒɪ], [tʃɪ]]",[dʒ],sonora,[ɪ],feren[tʃɪ],[dʒ] comeco
3,inconscien[tɪ]men[tɪ],"[[t], [tɪ]]",[t],surda,[ɪ],inconscienmen[tɪ],[t] meio
4,tris[te]men[te],"[[t], [te]]",[t],surda,[e],trismen[te],[t] meio
5,inocen[te]men[tɪ],"[[t], [tɪ]]",[t],surda,[e],inocenmen[tɪ],[t] meio
6,es[tɪ]ve[ɹ(v)][de][s],"[[t], [de]]",[t],surda,[ɪ],esve[ɹ(v)][de][s],[t] meio
7,tris[te]mente,[[t]],[t],surda,[e],trismente,[t] meio
8,[de],[[de]],[de],sonora,[e],,[de]único
9,[dɪ],[[dɪ]],[dɪ],sonora,[ɪ],,[dɪ]único


In [39]:
df = apply_cols(file_path)
df = df.apply(create_posicao_patalizacao_final, axis=1)

In [52]:
def df_add_duplicated_matches(file_path):
    df = apply_cols(file_path)
    df = df.apply(create_posicao_patalizacao_final, axis=1)

    # Regular expression pattern
    pattern = r'\[[^\]]*(t(?!ʃ)|tʃ|dʒ|de|dɪ)[^\]]*\]'

    
    # Find matches for each value in the column
    matches = df['palavra'].str.findall(pattern)
    
    # idenfitfy idexes where there is a match
    match_index = df[matches.apply(lambda x: len(x) > 1)].index
    
    #turn index object to list
    match_list = list(match_index)
    
    i = 0
    for element in match_list:

        element_used = element + i
        new_row_data = df.loc[element_used:element_used]
        new_row_data = new_row_data[['palavra','copy_palavra', 'patalizacao_cru']]

        df = concat([df.iloc[:element_used + 1], new_row_data, df.iloc[element_used + 1:]]).reset_index(drop=True)
        i = i + 1
    
    df = df.fillna('') 
    return df
# pd.set_option('display.maxs_rows', None)
df = df_add_duplicated_matches(file_path)
df

,palavra,patalizacao_cru,patalização,sonoridade,vogais_alvo,copy_palavra,posição_patalização
0,test[dʒɪ],[[dʒɪ]],[dʒ],sonora,[ɪ],test,[dʒ] final
1,[dʒɪ]ficulda[dʒɪ],"[[dʒɪ], [dʒɪ]]",[dʒ],sonora,[ɪ],ficulda[dʒɪ],[dʒ] comeco
2,[dʒɪ]ficulda[dʒɪ],"[[dʒɪ], [dʒɪ]]",,,,ficulda[dʒɪ],
3,[dʒɪ]feren[tʃɪ],"[[dʒɪ], [tʃɪ]]",[dʒ],sonora,[ɪ],feren[tʃɪ],[dʒ] comeco
4,[dʒɪ]feren[tʃɪ],"[[dʒɪ], [tʃɪ]]",,,,feren[tʃɪ],
5,inconscien[tɪ]men[tɪ],"[[t], [tɪ]]",[t],surda,[ɪ],inconscienmen[tɪ],[t] meio
6,inconscien[tɪ]men[tɪ],"[[t], [tɪ]]",,,,inconscienmen[tɪ],
7,tris[te]men[te],"[[t], [te]]",[t],surda,[e],trismen[te],[t] meio
8,tris[te]men[te],"[[t], [te]]",,,,trismen[te],
9,inocen[te]men[tɪ],"[[t], [tɪ]]",[t],surda,[e],inocenmen[tɪ],[t] meio


In [53]:
def new_create_patalizacao(row, i):
    
    pattern_t = r'\[[^\]]*t(?!ʃ)[^\]]*\]'
    pattern_tf = r'\[[^\]]*tʃ[^\]]*\]'
    pattern_dz = r'\[[^\]]*dʒ[^\]]*\]'
    pattern_de = r'\[[^\]]*de[^\]]*\]'
    pattern_di = r'\[[^\]]*dɪ[^\]]*\]'

    pattern = pattern_t + '|' + pattern_tf + '|' + pattern_dz + '|' + pattern_de + '|' + pattern_di
    
    
    patalizacao_cru = re.findall(pattern, row['palavra'])

    if row['patalização'] == '':
        if re.search(pattern_t, patalizacao_cru[i]):
            return '[t]'
        if re.search(pattern_tf, patalizacao_cru[i]):
            return '[tʃ]'
        if re.search(pattern_dz, patalizacao_cru[i]):
            return '[dʒ]'
        if re.search(pattern_de, patalizacao_cru[i]):
            return '[de]'
        if re.search(pattern_di, patalizacao_cru[i]):
            return '[dɪ]'

    return row['patalização']

# df['patalização'] = df.apply(new_create_patalizacao, axis=1)
df['patalização'] = df.apply(lambda row: new_create_patalizacao(row, 1), axis=1)
df

,palavra,patalizacao_cru,patalização,sonoridade,vogais_alvo,copy_palavra,posição_patalização
0,test[dʒɪ],[[dʒɪ]],[dʒ],sonora,[ɪ],test,[dʒ] final
1,[dʒɪ]ficulda[dʒɪ],"[[dʒɪ], [dʒɪ]]",[dʒ],sonora,[ɪ],ficulda[dʒɪ],[dʒ] comeco
2,[dʒɪ]ficulda[dʒɪ],"[[dʒɪ], [dʒɪ]]",[dʒ],,,ficulda[dʒɪ],
3,[dʒɪ]feren[tʃɪ],"[[dʒɪ], [tʃɪ]]",[dʒ],sonora,[ɪ],feren[tʃɪ],[dʒ] comeco
4,[dʒɪ]feren[tʃɪ],"[[dʒɪ], [tʃɪ]]",[tʃ],,,feren[tʃɪ],
5,inconscien[tɪ]men[tɪ],"[[t], [tɪ]]",[t],surda,[ɪ],inconscienmen[tɪ],[t] meio
6,inconscien[tɪ]men[tɪ],"[[t], [tɪ]]",[t],,,inconscienmen[tɪ],
7,tris[te]men[te],"[[t], [te]]",[t],surda,[e],trismen[te],[t] meio
8,tris[te]men[te],"[[t], [te]]",[t],,,trismen[te],
9,inocen[te]men[tɪ],"[[t], [tɪ]]",[t],surda,[e],inocenmen[tɪ],[t] meio


In [51]:
df

,palavra,patalizacao_cru,patalização,sonoridade,vogais_alvo,copy_palavra,posição_patalização
0,test[dʒɪ],[[dʒɪ]],[dʒ],sonora,[ɪ],test,[dʒ] final
1,[dʒɪ]ficulda[dʒɪ],"[[dʒɪ], [dʒɪ]]",[dʒ],sonora,[ɪ],ficulda[dʒɪ],[dʒ] comeco
2,[dʒɪ]ficulda[dʒɪ],"[[dʒɪ], [dʒɪ]]",[dʒ],,,ficulda[dʒɪ],
3,[dʒɪ]feren[tʃɪ],"[[dʒɪ], [tʃɪ]]",[dʒ],sonora,[ɪ],feren[tʃɪ],[dʒ] comeco
4,[dʒɪ]feren[tʃɪ],"[[dʒɪ], [tʃɪ]]",[tʃ],,,feren[tʃɪ],
5,inconscien[tɪ]men[tɪ],"[[t], [tɪ]]",[t],surda,[ɪ],inconscienmen[tɪ],[t] meio
6,inconscien[tɪ]men[tɪ],"[[t], [tɪ]]",[t],,,inconscienmen[tɪ],
7,tris[te]men[te],"[[t], [te]]",[t],surda,[e],trismen[te],[t] meio
8,tris[te]men[te],"[[t], [te]]",[t],,,trismen[te],
9,inocen[te]men[tɪ],"[[t], [tɪ]]",[t],surda,[e],inocenmen[tɪ],[t] meio


In [ ]:
df.drop(columns=['patalizacao_cru', 'copy_palavra'])

In [ ]:
 pattern_t = r'\[[^\]]*t(?!ʃ)[^\]]*\]'
pattern_tf = r'\[[^\]]*tʃ[^\]]*\]'
pattern_dz = r'\[[^\]]*dʒ[^\]]*\]'
pattern_de = r'\[[^\]]*de[^\]]*\]'
pattern_di = r'\[[^\]]*dɪ[^\]]*\]'

pattern = pattern_t + '|' + pattern_tf + '|' + pattern_dz + '|' + pattern_de + '|' + pattern_di
    
patalizacao_cru = re.findall(pattern, row['palavra'])
if len(patalizacao_cru) >= 2:
   
    
    # idenfitfy idexes where there is a match
    match_index = df[matches.apply(lambda x: len(x) > 1)].index
    